<a href="https://colab.research.google.com/github/IlyaKuprik/skolkovo-hack/blob/main/preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Process

## загрузка


In [ ]:
{"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

In [4]:
data_candidates_work_places = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_work_places.csv', sep=';',
                                names = ['CandidateId', 'Position','FromYear', 'FromMonth', 'ToYear', 'ToMonth'])

In [5]:
data_jobs = pd.read_csv('/content/drive/MyDrive/hack/data/data_jobs.csv', sep=';',
                                names = ['JobId', 'Status','Name', 'Region', 'Description'])

In [6]:
data_candidates = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates.csv', sep=';',
                                names = ['CandidateId', 'Position','Sex', 'Citizenship', 'Age',
                                'Salary', 'Langs', 'DriverLicense', 'Subway', 'Skills', 'Employment', 'Shedule',                                           
                                 'CandidateRegion', 'DateCreated', 'JobId', 'CandidateStatusId', 'Status'])

In [7]:
data_candidates_education = pd.read_csv('/content/drive/MyDrive/hack/data/data_candidates_education.csv', sep=';',
                                names = ['CandidateId', 'University', 'Faculty', 'GraduateYear'])

## группировка и создание последнего статуса

In [71]:
import re
 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});|-|/|;|_|•')
 
def cleanhtml(raw_html):
    if type(raw_html) == type(''):
        return re.sub(CLEANR, '', raw_html)
    return raw_html
 
data_jobs.Description = data_jobs.Description.apply(cleanhtml)

In [72]:
cols_val = {str(i): 'last' for i in data_candidates.columns}

In [73]:
grouped_with_last_status = data_candidates.sort_values(by = 'DateCreated').loc[:,data_candidates.columns]\
.groupby(by = ['CandidateId','JobId'], axis = 0, sort = False, as_index = False).agg( cols_val)

In [74]:
target_map = {"Самоотказ РЦ": 1, "Не отвечает": 0.1, "Отказ РЦ": 0, "Думает РЦ": 0.3, "Не пришел": 0.7, "Приглашен": 0.85, "Отклонен": 0.5, "Не пришёл": 0.8, 
  "Назначено собеседование в ресторан": 0.8, "Телефонное интервью": 0.7, "Пришел": 0.5, "Обзвон": 0.6, "ОТКАЗ СЦ": 0.3, "Отказ кандидата": 1, 
  "Отклик": 0.9, "Думает МП":0.4, "Расторгнул договор": 0.15, "Лендинг": 0.6, "Резерв": 0.5, "Отказ": 0.3, 'Вышел на линию': 0.9,
  "Принят на работу": 0.95, "Оформлен": 0.9, "Неактуально": 0, "Отказ компании": 0, "Перезаписался": 0.4, "Отказался/Нашел работу": 0.3,
  "Заключил договор": 0.8, "Отправлен на СБ": 0.85, "Интервью с HR": 0.85, "Самоотказ МП": 0.5, "Недозвон": 0.2, "Отказ по итогам скрининга резюме": 0,
  "Не прошёл СБ": 0.5, "Отказ HR": 0, "Резерв РЦ": 0.2, "Не пришел (повторно)": 0.6, "Интервью": 0.8, "Отказ ВВСС (другое)": 0.2, "Отказ СБ": 0.65}

## Штрафы

In [75]:
targets = grouped_with_last_status['Status'].apply(lambda x: target_map[x] if x in target_map else 1/2) ## Заполнили NA
grouped_with_ids_target = grouped_with_last_status.loc[:, ['CandidateId', 'JobId',]]
grouped_with_ids_target['target'] = targets

In [76]:
#Инициализация штрафов
number_fields = ['Age', 'Salary']
string_fields = ['Position', 'Citizenship', 'Langs',\
                  'Skills', 'Employment', 'Shedule', 'CandidateRegion']
fines_vector = [1/18]*9

In [77]:

mask_df = grouped_with_last_status.copy()
mask_df[number_fields] = grouped_with_last_status[number_fields].apply(lambda x: x == 0)
mask_df[string_fields] = grouped_with_last_status[string_fields].apply(lambda x: x.isna())

total_fine = mask_df[number_fields + string_fields].apply(lambda x: 1 - (np.array(x) * np.array(fines_vector)).sum(), axis = 1)
total_target = targets*total_fine
grouped_with_ids_target['target'] = total_target

## Обработка текста

In [78]:
grouped_table_candidates_features = grouped_with_last_status.drop(columns = ['Status', 'CandidateStatusId'])

In [ ]:
grouped_table_candidates_features

In [79]:
new_df = grouped_table_candidates_features\
    .merge(data_jobs, on='JobId')\
    .merge(data_candidates_education.drop_duplicates('CandidateId'), on='CandidateId', how='left')
new_df.index = [new_df.CandidateId, new_df.JobId]
df = new_df.drop(columns=['Region', 'Description', 'JobId', 'CandidateId'])
df.Sex = df.Sex\
    .where(~(df.Sex == 2), other='мужчина')\
    .where(~(df.Sex == 1), other='женщина')\
    .where(~(df.Sex == 0), other='неопределен')
df = df.fillna('NaN').astype(str).drop(columns = ["Status"]).add('. ').sum(axis=1).reset_index()
 
status_jobs = pd.DataFrame([[1, 'открытая'], [2, 'закрытая'], [3, 'приостановленная'], [4, 'отменена'], [5, 'черновик']],
                          columns = ['Status', 'name_status'])
new_data_jobs = data_jobs\
    .merge(status_jobs, on='Status')
new_data_jobs.index = new_data_jobs.JobId
job_user_tokens1 = new_data_jobs\
    .drop(columns=['Status', 'JobId'])\
    .fillna('NaN')\
    .astype(str)\
    .add('. ')\
    .sum(axis=1)\
    .reset_index()
 
job_user_tokens2 = df
job_user_tokens_with_id = job_user_tokens2\
    .merge(job_user_tokens1, on='JobId')

In [62]:
merged_descriptions_targets_ids = job_user_tokens_with_id.merge(grouped_with_ids_target, on = ['JobId', 'CandidateId'], how = 'inner')
merged_descriptions_targets_ids.columns = ["CandidateId", "JobId", "Candidate_descr", "Jobs_descr", "target"]
merged_descriptions_targets_ids.isna().sum()
merged_descriptions_targets_ids = merged_descriptions_targets_ids.fillna(1/2)

In [63]:
merged_descriptions_targets_ids

,CandidateId,JobId,Candidate_descr,Jobs_descr,target
0,23669,369,Водитель. мужчина. Гражданство. 37. 0. NaN. Na...,Водитель. Санкт-Петербург. Требования к опыту...,0.50
1,23954,369,Водитель. неопределен. NaN. 0. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. Требования к опыту...,0.95
2,23680,369,Водитель. мужчина. Россия. 27. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. Требования к опыту...,0.50
3,23682,369,Водитель. мужчина. Россия. 25. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. Требования к опыту...,0.50
4,23681,369,Водитель. мужчина. Россия. 35. 0. NaN. NaN. Na...,Водитель. Санкт-Петербург. Требования к опыту...,0.50
...,...,...,...,...,...
57353,24191092,370097,Водитель ричтрака (погрузчика) на складе катег...,Водитель погрузчика / штабелера. Москва. Водит...,0.50
57354,24191343,370097,Водитель погрузчика. мужчина. Россия. 48. 8000...,Водитель погрузчика / штабелера. Москва. Водит...,0.00
57355,24207766,370097,Кладовщик-комплектовщик. мужчина. Россия. 35. ...,Водитель погрузчика / штабелера. Москва. Водит...,0.70
57356,24207796,370097,"Водитель погрузчика, Штабелера. мужчина. Росси...",Водитель погрузчика / штабелера. Москва. Водит...,0.70


In [80]:
merged_descriptions_targets_ids.to_csv('preprocess_data.csv')